# 0. Set-Ups

General hints for this notebook:
- Spark UI usually accesible by http://localhost:4040/ or http://localhost:4041/
- Deep dive Spark ui happens in later episodes
- sc.setJobDescription("Description") replaces the Job Description of an action in the Spark UI with your own
- sdf.rdd.getNumPartitions() returns the number partitions of the current Spark DataFrame
- sdf.write.format("noop").mode("overwrite").save() is a good way to analyze and initiate actions for transformations without side effects during an actual write

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import pyspark

In [2]:
spark = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")
#to not cache datafrimes... this may not create repeatable results
spark.conf.set("spark.databricks.io.cache.enabled", "false")

In [4]:
d = [
    {"a":"a", "b": 1},
    {"a":"b", "b": 2},
    {"a":"c", "b": 3},
    {"a":"d", "b": 4},
    {"a":"e", "b": 5},
    {"a":"e", "b": 6},
    {"a":"f", "b": 7},
    {"a":"g", "b": 8},
    {"a":"h", "b": 9},
    {"a":"i", "b": 10},
    {"a":"j", "b": 11},
    {"a":"k", "b": 12},
    {"a":"a", "b": 13},
    {"a":"b", "b": 13},
]
ddl_schema = "a string, b int"
sdf = spark.createDataFrame(d, schema=ddl_schema)

In [5]:
sdf.rdd.getNumPartitions()

4

# 1. Lazy Execution and actions

You can see here the following things:
- The simple filter operation runs within ms
- In the Spark UI under jobs you will not see any job when running only the filter operation
- Only once you run a Job as here count or later write Spark becomes active and runs the actual calculation
- Once count is executed you will see the Job in the Spark UI
- The data has been paritioned into 4 partitions (more in the next episodes)

In [6]:
sdf_lazy = sdf.filter(f.col("b") > 5)

In [7]:
sc.setJobDescription("LazyExecution")
sdf_lazy.count()

9

In [8]:
sdf_lazy.rdd.getNumPartitions()

4

# 2. Noop write

- The same behavior as before. Write is an action and Spark is running a job. 
- We use a noop write which does not have any side effects and spark optimizations while executing an action

In [9]:
sc.setJobDescription("NoopWrite")
sdf.write.format("noop").mode("overwrite").save()

# 3. Narrow transformation with noop write
- A narrow transformation as you hopefully already know does not have any shuffle operations
- This means if no Spark or Databricks optimizations active we will see only one Job with one Stage and the number of tasks related to the Spark partition size of the data.
- Hint: the number of stages can also vary depending on the action. E.g. with count as you see next. That's why a noop write is great to analyze without side effects
- Another indicator is the number partitions before and after the transformation which does not change.

In [10]:
sc.setJobDescription("FilterNoopWrite")
sdf_narrowNoop = sdf.filter(f.col("b") > 5)
sdf_narrowNoop.write.format("noop").mode("overwrite").save()

In [19]:
sdf_narrowNoop.rdd.getNumPartitions()

4

# 4. Count
- We see that the count creates two stages
- The first stage has the same number of tasks as before. Here on each executer the partial counts of each partitions are calculated
- Afterwards we have an exchange of all those informations into one last executer to caclualte the final count. That's why the second stage only has one task

In [11]:
sc.setJobDescription("Count")
sdf.count()

14

# 5. Wide transformation
- For wide transformations we have usually shuffle operations and an exchange for the data and thus two stages similar to a count. Those operations often are reason for unefficiencies. 
- A hint for a wide transformation is the change of partitions as the shuffle creates a repartioning during this process
- You can see that the number of partitions depends on the value of "spark.sql.shuffle.partitions". Default 200
- In the Spark UI under SQL and then Details we can see in the physical plan the Hash partitioning which usually happens during re-shuffling the data

In [12]:
#Turning off AQE as it generates more jobs which might be confusing for this scenario here. 
spark.conf.set("spark.sql.adaptive.enabled", "false")

sc.setJobDescription("Wide")
sdf_w = sdf.groupBy("a").count()
sdf_w.write.format("noop").mode("overwrite").save()

In [13]:
sc.setJobDescription("WideShow")
sdf_w.show()

+---+-----+
|  a|count|
+---+-----+
|  g|    1|
|  f|    1|
|  k|    1|
|  e|    2|
|  h|    1|
|  d|    1|
|  c|    1|
|  i|    1|
|  j|    1|
|  b|    2|
|  a|    2|
+---+-----+



In [14]:
sdf_w.rdd.getNumPartitions()

200

In [15]:
spark.conf.get("spark.sql.shuffle.partitions")

'200'

# 6. Wide Transformations with AQE
- Similar to before we run a wide transformation here just to showcase why we turned off AQE
- You probably already assumed that for 14 rows 200 partitions is not benefitial
- One of AQEs feature is to coalesce a lot of small partitions into bigger once. In this case one partion is left at the end
- This can be seen in the Job DAG visualisation in the second stage as an AQE Shuffle Read

In [16]:
#Turning on AQE
spark.conf.set("spark.sql.adaptive.enabled", "true")

sc.setJobDescription("WideAQE")
sdf_w_aqe = sdf.groupBy("a").count()
sdf_w_aqe.write.format("noop").mode("overwrite").save()

In [17]:
sc.setJobDescription("WideShowAQE")
sdf_w_aqe.show()

+---+-----+
|  a|count|
+---+-----+
|  c|    1|
|  b|    2|
|  a|    2|
|  e|    2|
|  d|    1|
|  g|    1|
|  f|    1|
|  h|    1|
|  k|    1|
|  i|    1|
|  j|    1|
+---+-----+



In [18]:
sdf_w_aqe.rdd.getNumPartitions()

1